# 向量嵌入和检索

## BM25

In [10]:
from collections import Counter
import math
import numpy as np
from typing import List, Dict, Tuple


class BM25Retriever:
    """
    BM25 (Best Matching 25) 检索器
    
    BM25 是一种基于概率的信息检索算法，广泛应用于搜索引擎和文档检索系统。
    它通过计算词频 (TF) 和逆文档频率 (IDF) 来评估文档与查询的相关性。
    
    参数:
        k1: 控制词频饱和度的参数，通常取值 1.2-2.0，默认 1.5
        b: 控制文档长度归一化的参数，取值 0-1，默认 0.75
    """
    
    def __init__(self, k1: float = 1.5, b: float = 0.75):
        self.k1 = k1
        self.b = b
        self.documents = []
        self.vocabulary = set()
        self.vocab_to_idx = {}
        self.idf = {}
        self.avg_doc_len = 0
        self.doc_embeddings = []
        
    def fit(self, documents: List[str], delimiter: str = "，"):
        """
        训练 BM25 模型
        
        参数:
            documents: 文档列表
            delimiter: 分词分隔符
        """
        self.documents = documents
        N = len(documents)
        
        # 构建词表
        self.vocabulary = set(word for doc in documents for word in doc.split(delimiter))
        self.vocab_to_idx = {word: idx for idx, word in enumerate(sorted(self.vocabulary))}
        
        # 计算文档频率 (DF)
        df = Counter()
        for doc in documents:
            unique_words = set(doc.split(delimiter))
            for word in unique_words:
                df[word] += 1
        
        # 计算逆文档频率 (IDF)
        # IDF(w) = log((N - df(w) + 0.5) / (df(w) + 0.5) + 1)
        self.idf = {
            word: math.log((N - df[word] + 0.5) / (df[word] + 0.5) + 1) 
            for word in self.vocabulary
        }
        
        # 计算平均文档长度
        self.avg_doc_len = sum(len(doc.split(delimiter)) for doc in documents) / N
        
        # 预计算所有文档的 BM25 嵌入
        self.doc_embeddings = [self._compute_embedding(doc, delimiter) for doc in documents]
        
        print(f"✓ BM25 模型训练完成")
        print(f"  - 文档数量: {N}")
        print(f"  - 词表大小: {len(self.vocabulary)}")
        print(f"  - 平均文档长度: {self.avg_doc_len:.2f}")
        print(f"  - 参数: k1={self.k1}, b={self.b}")
        
    def _compute_embedding(self, text: str, delimiter: str = "，") -> Dict[int, float]:
        """
        计算文本的 BM25 稀疏嵌入
        
        BM25 公式:
        score(w) = IDF(w) × (TF(w) × (k1 + 1)) / (TF(w) + k1 × (1 - b + b × |D| / avgdl))
        
        其中:
        - TF(w): 词 w 在文档中的频率
        - IDF(w): 词 w 的逆文档频率
        - |D|: 当前文档长度
        - avgdl: 平均文档长度
        """
        tf = Counter(text.split(delimiter))
        doc_len = len(text.split(delimiter))
        embedding = {}
        
        for word, freq in tf.items():
            if word in self.vocabulary:
                idx = self.vocab_to_idx[word]
                # BM25 计分公式
                numerator = self.idf[word] * freq * (self.k1 + 1)
                denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.avg_doc_len)
                score = numerator / denominator
                embedding[idx] = score
                
        return embedding
    
    def search(self, query: str, top_k: int = 3, delimiter: str = "，") -> List[Tuple[int, float, str]]:
        """
        使用 BM25 检索最相关的文档
        
        参数:
            query: 查询文本
            top_k: 返回前 k 个最相关的文档
            delimiter: 分词分隔符
            
        返回:
            List of (文档索引, 相关性分数, 文档内容)
        """
        # 计算查询的 BM25 嵌入
        query_embedding = self._compute_embedding(query, delimiter)
        
        # 计算查询与每个文档的相关性分数
        scores = []
        for doc_idx, doc_embedding in enumerate(self.doc_embeddings):
            # 计算两个稀疏向量的点积（共同词项的分数相乘相加）
            score = sum(
                query_embedding.get(idx, 0) * doc_embedding.get(idx, 0)
                for idx in set(query_embedding.keys()) | set(doc_embedding.keys())
            )
            scores.append((doc_idx, score, self.documents[doc_idx]))
        
        # 按分数降序排序并返回 top_k
        scores.sort(key=lambda x: x[1], reverse=True)
        return scores[:top_k]
    
    def get_word_importance(self, doc_idx: int, top_n: int = 5) -> List[Tuple[str, float]]:
        """
        获取文档中最重要的词及其 BM25 分数
        
        参数:
            doc_idx: 文档索引
            top_n: 返回前 n 个重要词
            
        返回:
            List of (词, BM25分数)
        """
        idx_to_vocab = {idx: word for word, idx in self.vocab_to_idx.items()}
        embedding = self.doc_embeddings[doc_idx]
        
        # 按分数排序
        word_scores = [(idx_to_vocab[idx], score) for idx, score in embedding.items()]
        word_scores.sort(key=lambda x: x[1], reverse=True)
        
        return word_scores[:top_n]
    
    def print_embeddings(self, show_top_words: int = 5):
        """打印所有文档的嵌入信息"""
        print(f"\n{'='*80}")
        print(f"BM25 稀疏嵌入详情")
        print(f"{'='*80}")
        
        for idx, (doc, embedding) in enumerate(zip(self.documents, self.doc_embeddings)):
            print(f"\n📄 文档 {idx + 1}:")
            print(f"   内容: {doc}")
            print(f"   嵌入维度: {len(self.vocabulary)} (稀疏度: {len(embedding)}/{len(self.vocabulary)})")
            print(f"   非零元素: {len(embedding)}")
            
            # 显示最重要的词
            top_words = self.get_word_importance(idx, show_top_words)
            print(f"   Top-{show_top_words} 关键词:")
            for word, score in top_words:
                print(f"     - {word}: {score:.4f}")


def main():
    """主函数：演示 BM25 的使用"""
    
    # 猢狲的战斗日志数据集
    battle_logs = [
        "猢狲，施展，烈焰拳，击退，妖怪；随后开启，金刚体，抵挡，神兵，攻击。",
        "妖怪，使用，寒冰箭，攻击，猢狲，但被，烈焰拳，反击，击溃。",
        "猢狲，召唤，烈焰拳，与，毁灭咆哮，击败，妖怪，随后，收集，妖怪，精华。",
        "猢狲，使用，金刚体，防御，敌人，攻击，成功，保护，队友。",
        "战斗，结束，猢狲，获得，经验值，与，战利品，提升，等级。"
    ]
    
    print("🔥 BM25 稀疏嵌入与检索演示")
    print("="*80)
    
    # 1. 创建并训练 BM25 检索器
    print("\n【步骤 1】训练 BM25 模型")
    print("-"*80)
    retriever = BM25Retriever(k1=1.5, b=0.75)
    retriever.fit(battle_logs)
    
    # 2. 打印所有文档的嵌入
    retriever.print_embeddings(show_top_words=5)
    
    # 3. 执行检索查询
    print(f"\n{'='*80}")
    print("【步骤 2】执行检索查询")
    print(f"{'='*80}")
    
    queries = [
        "猢狲，烈焰拳，攻击",
        "金刚体，防御",
        "妖怪，精华"
    ]
    
    for query in queries:
        print(f"\n🔍 查询: {query}")
        print("-"*80)
        results = retriever.search(query, top_k=3)
        
        for rank, (doc_idx, score, doc_content) in enumerate(results, 1):
            print(f"  {rank}. [文档 {doc_idx + 1}] 相关性分数: {score:.4f}")
            print(f"     内容: {doc_content}")
    
    # 4. 词重要性分析
    print(f"\n{'='*80}")
    print("【步骤 3】词重要性分析 (IDF 值)")
    print(f"{'='*80}")
    
    # 显示 IDF 最高的词（最具区分性的词）
    sorted_idf = sorted(retriever.idf.items(), key=lambda x: x[1], reverse=True)
    print("\nTop-10 最具区分性的词 (高 IDF):")
    for word, idf_score in sorted_idf[:10]:
        print(f"  - {word}: {idf_score:.4f}")
    
    # 5. 性能统计
    print(f"\n{'='*80}")
    print("【步骤 4】性能统计")
    print(f"{'='*80}")
    total_elements = sum(len(emb) for emb in retriever.doc_embeddings)
    max_possible = len(retriever.documents) * len(retriever.vocabulary)
    sparsity = (1 - total_elements / max_possible) * 100
    
    print(f"  总非零元素数: {total_elements}")
    print(f"  最大可能元素数: {max_possible}")
    print(f"  稀疏度: {sparsity:.2f}%")
    print(f"  平均每文档非零词数: {total_elements / len(retriever.documents):.2f}")


if __name__ == '__main__':
    main()

🔥 BM25 稀疏嵌入与检索演示

【步骤 1】训练 BM25 模型
--------------------------------------------------------------------------------
✓ BM25 模型训练完成
  - 文档数量: 5
  - 词表大小: 35
  - 平均文档长度: 9.40
  - 参数: k1=1.5, b=0.75

BM25 稀疏嵌入详情

📄 文档 1:
   内容: 猢狲，施展，烈焰拳，击退，妖怪；随后开启，金刚体，抵挡，神兵，攻击。
   嵌入维度: 35 (稀疏度: 9/35)
   非零元素: 9
   Top-5 关键词:
     - 施展: 1.4134
     - 击退: 1.4134
     - 妖怪；随后开启: 1.4134
     - 抵挡: 1.4134
     - 神兵: 1.4134

📄 文档 2:
   内容: 妖怪，使用，寒冰箭，攻击，猢狲，但被，烈焰拳，反击，击溃。
   嵌入维度: 35 (稀疏度: 9/35)
   非零元素: 9
   Top-5 关键词:
     - 寒冰箭: 1.4134
     - 但被: 1.4134
     - 反击: 1.4134
     - 击溃。: 1.4134
     - 妖怪: 0.8926

📄 文档 3:
   内容: 猢狲，召唤，烈焰拳，与，毁灭咆哮，击败，妖怪，随后，收集，妖怪，精华。
   嵌入维度: 35 (稀疏度: 10/35)
   非零元素: 10
   Top-5 关键词:
     - 召唤: 1.2877
     - 毁灭咆哮: 1.2877
     - 击败: 1.2877
     - 随后: 1.2877
     - 收集: 1.2877

📄 文档 4:
   内容: 猢狲，使用，金刚体，防御，敌人，攻击，成功，保护，队友。
   嵌入维度: 35 (稀疏度: 9/35)
   非零元素: 9
   Top-5 关键词:
     - 防御: 1.4134
     - 敌人: 1.4134
     - 成功: 1.4134
     - 保护: 1.4134
     - 队友。: 1.4134

📄 文档 5:
   内容: 战斗，结束，猢狲，获得，经验

## BGE-M3

In [11]:
import os
import sys

# 临时绕过 torch.load 安全检查（仅用于开发环境）
# 注意：这是一个临时解决方案，生产环境应该升级 torch 到 2.6+ 或使用 safetensors 格式
import transformers.utils.import_utils
import transformers.modeling_utils

def dummy_check():
    pass

transformers.utils.import_utils.check_torch_load_is_safe = dummy_check
transformers.modeling_utils.check_torch_load_is_safe = dummy_check

from FlagEmbedding import BGEM3FlagModel
import numpy as np
from typing import List, Dict, Tuple, Optional
import time


class BGEM3Retriever:
    """
    BGE-M3 混合检索器
    
    BGE-M3 (BAAI General Embedding - Multi-Lingual, Multi-Functionality, Multi-Granularity)
    是一个强大的多语言嵌入模型，支持三种检索方式：
    
    1. 密集检索 (Dense Retrieval): 语义相似度匹配，适合捕捉深层语义
    2. 稀疏检索 (Sparse Retrieval): 基于词汇匹配，类似 BM25，适合精确匹配
    3. 多向量检索 (Multi-Vector Retrieval): ColBERT 风格，细粒度交互匹配
    
    参数:
        model_name: 模型名称，默认 "BAAI/bge-m3"
        use_fp16: 是否使用半精度浮点数，默认 False
    """
    
    def __init__(self, model_name: str = "BAAI/bge-m3", use_fp16: bool = False):
        print(f"🔧 正在加载 BGE-M3 模型: {model_name}")
        start_time = time.time()
        
        self.model = BGEM3FlagModel(model_name, use_fp16=use_fp16)
        self.documents = []
        self.dense_embeddings = None
        self.sparse_embeddings = None
        self.colbert_embeddings = None
        
        load_time = time.time() - start_time
        print(f"✓ 模型加载完成 (耗时: {load_time:.2f}秒)")
    
    def encode(self, texts: List[str], batch_size: int = 12, max_length: int = 512) -> Dict:
        """
        编码文本，获取三种嵌入
        
        参数:
            texts: 文本列表
            batch_size: 批处理大小
            max_length: 最大序列长度
            
        返回:
            包含三种嵌入的字典
        """
        print(f"  📝 编码 {len(texts)} 个文本...")
        start_time = time.time()
        
        embeddings = self.model.encode(
            texts,
            batch_size=batch_size,
            max_length=max_length,
            return_dense=True,
            return_sparse=True,
            return_colbert_vecs=True
        )
        
        encode_time = time.time() - start_time
        print(f"  ✓ 编码完成 (耗时: {encode_time:.2f}秒, 速度: {len(texts)/encode_time:.2f} docs/s)")
        
        return embeddings
    
    def fit(self, documents: List[str]):
        """
        索引文档集合
        
        参数:
            documents: 文档列表
        """
        print(f"\n{'='*80}")
        print(f"【步骤 1】索引文档")
        print(f"{'='*80}")
        print(f"文档数量: {len(documents)}")
        
        self.documents = documents
        embeddings = self.encode(documents)
        
        self.dense_embeddings = embeddings['dense_vecs']
        self.sparse_embeddings = embeddings['lexical_weights']
        self.colbert_embeddings = embeddings['colbert_vecs']
        
        # 统计信息
        avg_sparse_len = np.mean([len(emb) for emb in self.sparse_embeddings])
        avg_colbert_len = np.mean([emb.shape[0] for emb in self.colbert_embeddings])
        
        print(f"\n📊 索引统计:")
        print(f"  - 密集嵌入维度: {self.dense_embeddings[0].shape[0]}")
        print(f"  - 平均稀疏嵌入非零元素: {avg_sparse_len:.2f}")
        print(f"  - 平均多向量数量: {avg_colbert_len:.2f}")
    
    def search_dense(self, query: str, top_k: int = 3) -> List[Tuple[int, float]]:
        """
        密集检索：使用余弦相似度进行语义匹配
        
        优势：捕捉深层语义，支持跨语言检索
        """
        query_embedding = self.encode([query])
        query_dense = query_embedding['dense_vecs'][0]
        
        # 计算余弦相似度
        scores = []
        for idx, doc_dense in enumerate(self.dense_embeddings):
            # 余弦相似度 = dot(A, B) / (norm(A) * norm(B))
            similarity = np.dot(query_dense, doc_dense) / (
                np.linalg.norm(query_dense) * np.linalg.norm(doc_dense)
            )
            scores.append((idx, float(similarity)))
        
        # 排序并返回 top_k
        scores.sort(key=lambda x: x[1], reverse=True)
        return scores[:top_k]
    
    def search_sparse(self, query: str, top_k: int = 3) -> List[Tuple[int, float]]:
        """
        稀疏检索：基于词汇匹配，类似 BM25
        
        优势：精确词汇匹配，可解释性强
        """
        query_embedding = self.encode([query])
        query_sparse = query_embedding['lexical_weights'][0]
        
        # 计算稀疏向量点积
        scores = []
        for idx, doc_sparse in enumerate(self.sparse_embeddings):
            # 点积：共同词项的权重相乘相加
            score = sum(
                query_sparse.get(key, 0) * doc_sparse.get(key, 0)
                for key in set(query_sparse.keys()) | set(doc_sparse.keys())
            )
            scores.append((idx, float(score)))
        
        scores.sort(key=lambda x: x[1], reverse=True)
        return scores[:top_k]
    
    def search_colbert(self, query: str, top_k: int = 3) -> List[Tuple[int, float]]:
        """
        多向量检索：ColBERT 风格的细粒度交互
        
        优势：token 级别的精细匹配，平衡语义和词汇
        """
        query_embedding = self.encode([query])
        query_colbert = query_embedding['colbert_vecs'][0]
        
        # MaxSim 操作：对每个查询 token，找到文档中最相似的 token
        scores = []
        for idx, doc_colbert in enumerate(self.colbert_embeddings):
            # 计算查询和文档之间的相似度矩阵
            sim_matrix = np.matmul(query_colbert, doc_colbert.T)
            # 对每个查询 token，取最大相似度，然后求和
            max_sim = np.max(sim_matrix, axis=1).sum()
            scores.append((idx, float(max_sim)))
        
        scores.sort(key=lambda x: x[1], reverse=True)
        return scores[:top_k]
    
    def search_hybrid(
        self, 
        query: str, 
        top_k: int = 3,
        weights: Optional[Dict[str, float]] = None
    ) -> List[Tuple[int, float]]:
        """
        混合检索：融合三种检索方式的结果
        
        参数:
            query: 查询文本
            top_k: 返回前 k 个结果
            weights: 三种检索方式的权重，默认 {'dense': 0.4, 'sparse': 0.3, 'colbert': 0.3}
        """
        if weights is None:
            weights = {'dense': 0.4, 'sparse': 0.3, 'colbert': 0.3}
        
        # 获取三种检索结果
        dense_results = self.search_dense(query, len(self.documents))
        sparse_results = self.search_sparse(query, len(self.documents))
        colbert_results = self.search_colbert(query, len(self.documents))
        
        # 归一化分数到 [0, 1]
        def normalize_scores(results):
            if not results or max(r[1] for r in results) == 0:
                return {idx: 0 for idx, _ in results}
            max_score = max(r[1] for r in results)
            min_score = min(r[1] for r in results)
            score_range = max_score - min_score if max_score != min_score else 1
            return {idx: (score - min_score) / score_range for idx, score in results}
        
        dense_norm = normalize_scores(dense_results)
        sparse_norm = normalize_scores(sparse_results)
        colbert_norm = normalize_scores(colbert_results)
        
        # 计算加权融合分数
        final_scores = []
        for idx in range(len(self.documents)):
            score = (
                weights['dense'] * dense_norm.get(idx, 0) +
                weights['sparse'] * sparse_norm.get(idx, 0) +
                weights['colbert'] * colbert_norm.get(idx, 0)
            )
            final_scores.append((idx, score))
        
        final_scores.sort(key=lambda x: x[1], reverse=True)
        return final_scores[:top_k]
    
    def compare_retrievals(self, query: str, top_k: int = 3):
        """
        比较三种检索方式的结果
        """
        print(f"\n{'='*80}")
        print(f"🔍 查询: {query}")
        print(f"{'='*80}")
        
        methods = [
            ('密集检索 (Dense)', self.search_dense, '🔷'),
            ('稀疏检索 (Sparse)', self.search_sparse, '🔶'),
            ('多向量检索 (ColBERT)', self.search_colbert, '🔸'),
            ('混合检索 (Hybrid)', self.search_hybrid, '⭐')
        ]
        
        for method_name, method_func, emoji in methods:
            print(f"\n{emoji} {method_name}")
            print("-" * 80)
            
            results = method_func(query, top_k)
            for rank, (doc_idx, score) in enumerate(results, 1):
                print(f"  {rank}. [文档 {doc_idx + 1}] 分数: {score:.4f}")
                print(f"     {self.documents[doc_idx]}")
    
    def analyze_embeddings(self, text: str):
        """
        分析单个文本的三种嵌入特性
        """
        print(f"\n{'='*80}")
        print(f"📊 嵌入特性分析")
        print(f"{'='*80}")
        print(f"文本: {text}")
        
        embeddings = self.encode([text])
        
        # 密集嵌入分析
        dense = embeddings['dense_vecs'][0]
        print(f"\n🔷 密集嵌入 (Dense Embedding)")
        print(f"  - 维度: {dense.shape}")
        print(f"  - L2 范数: {np.linalg.norm(dense):.4f}")
        print(f"  - 均值: {np.mean(dense):.4f}")
        print(f"  - 标准差: {np.std(dense):.4f}")
        print(f"  - 前10维: {dense[:10]}")
        
        # 稀疏嵌入分析
        sparse = embeddings['lexical_weights'][0]
        sorted_sparse = sorted(sparse.items(), key=lambda x: x[1], reverse=True)
        print(f"\n🔶 稀疏嵌入 (Sparse Embedding)")
        print(f"  - 非零元素数: {len(sparse)}")
        print(f"  - 总权重: {sum(sparse.values()):.4f}")
        print(f"  - 最大权重: {max(sparse.values()):.4f}")
        print(f"  - Top-10 词项ID及权重:")
        for token_id, weight in sorted_sparse[:10]:
            print(f"      ID {token_id}: {weight:.4f}")
        
        # 多向量嵌入分析
        colbert = embeddings['colbert_vecs'][0]
        print(f"\n🔸 多向量嵌入 (ColBERT Embedding)")
        print(f"  - 向量数量: {colbert.shape[0]}")
        print(f"  - 每个向量维度: {colbert.shape[1]}")
        print(f"  - 向量间平均余弦相似度: {self._avg_cosine_similarity(colbert):.4f}")
        print(f"  - 第1个向量前10维: {colbert[0][:10]}")
        print(f"  - 第2个向量前10维: {colbert[1][:10]}")
    
    def _avg_cosine_similarity(self, vectors):
        """计算向量集合的平均余弦相似度"""
        if len(vectors) < 2:
            return 0.0
        similarities = []
        for i in range(len(vectors)):
            for j in range(i + 1, len(vectors)):
                sim = np.dot(vectors[i], vectors[j]) / (
                    np.linalg.norm(vectors[i]) * np.linalg.norm(vectors[j])
                )
                similarities.append(sim)
        return np.mean(similarities)


def main():
    """主函数：演示 BGE-M3 的使用"""
    
    print("="*80)
    print("🚀 BGE-M3 多功能嵌入与混合检索演示")
    print("="*80)
    
    # 创建检索器
    retriever = BGEM3Retriever("BAAI/bge-m3", use_fp16=False)
    
    # 战斗日志数据集
    battle_logs = [
        "猢狲施展烈焰拳，击退妖怪；随后开启金刚体，抵挡神兵攻击。",
        "妖怪使用寒冰箭攻击猢狲，但被烈焰拳反击击溃。",
        "猢狲召唤烈焰拳与毁灭咆哮，击败妖怪，随后收集妖怪精华。",
        "猢狲使用金刚体防御敌人攻击，成功保护队友。",
        "战斗结束后，猢狲获得经验值与战利品，成功提升等级。",
        "在深渊副本中，猢狲遭遇强大的Boss，使用终极技能黑暗爆发。",
        "队友为猢狲提供治疗和增益效果，帮助其恢复生命值和法力值。"
    ]
    
    # 索引文档
    retriever.fit(battle_logs)
    
    # 分析单个文本的嵌入特性
    sample_text = "猢狲施展烈焰拳，击退妖怪；随后开启金刚体，抵挡神兵攻击。"
    retriever.analyze_embeddings(sample_text)
    
    # 比较不同检索方式
    print(f"\n{'='*80}")
    print(f"【步骤 2】多种检索方式对比")
    print(f"{'='*80}")
    
    queries = [
        "烈焰拳攻击妖怪",
        "金刚体防御技能",
        "战斗获得经验值",
        "队友治疗和增益"
    ]
    
    for query in queries:
        retriever.compare_retrievals(query, top_k=3)
    
    # 性能对比总结
    print(f"\n{'='*80}")
    print(f"【步骤 3】检索方式特点总结")
    print(f"{'='*80}")
    
    print("""
🔷 密集检索 (Dense Retrieval)
   优势: 语义理解能力强，支持跨语言检索，可以理解同义词和概念
   劣势: 对罕见词和专有名词支持较弱
   适用: 语义搜索、推荐系统、问答系统

🔶 稀疏检索 (Sparse Retrieval)
   优势: 精确词汇匹配，可解释性强，对关键词敏感
   劣势: 无法理解同义词和语义关系
   适用: 关键词搜索、专有名词检索、精确匹配场景

🔸 多向量检索 (ColBERT)
   优势: Token级别精细匹配，平衡语义和词汇，检索质量高
   劣势: 存储开销大，计算复杂度高
   适用: 高质量检索场景、需要细粒度匹配的任务

⭐ 混合检索 (Hybrid)
   优势: 结合多种检索优势，检索效果最稳定可靠
   劣势: 需要调整权重参数
   适用: 通用检索场景、生产环境推荐使用
    """)
    
    print(f"\n{'='*80}")
    print("✓ 演示完成！")
    print(f"{'='*80}")


if __name__ == '__main__':
    main()


🚀 BGE-M3 多功能嵌入与混合检索演示
🔧 正在加载 BGE-M3 模型: BAAI/bge-m3


Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 63009.07it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


✓ 模型加载完成 (耗时: 7.73秒)

【步骤 1】索引文档
文档数量: 7
  📝 编码 7 个文本...
  ✓ 编码完成 (耗时: 1.38秒, 速度: 5.06 docs/s)

📊 索引统计:
  - 密集嵌入维度: 1024
  - 平均稀疏嵌入非零元素: 16.71
  - 平均多向量数量: 21.14

📊 嵌入特性分析
文本: 猢狲施展烈焰拳，击退妖怪；随后开启金刚体，抵挡神兵攻击。
  📝 编码 1 个文本...
  ✓ 编码完成 (耗时: 0.48秒, 速度: 2.06 docs/s)

🔷 密集嵌入 (Dense Embedding)
  - 维度: (1024,)
  - L2 范数: 1.0000
  - 均值: -0.0007
  - 标准差: 0.0312
  - 前10维: [ 0.01156882  0.024392   -0.02763914 -0.00984303 -0.04426071 -0.02911931
  0.03953091  0.0216518   0.01074652 -0.03528602]

🔶 稀疏嵌入 (Sparse Embedding)
  - 非零元素数: 23
  - 总权重: 3.4500
  - 最大权重: 0.2537
  - Top-10 词项ID及权重:
      ID 75133: 0.2537
      ID 26471: 0.2490
      ID 19752: 0.2313
      ID 22477: 0.2181
      ID 4742: 0.2175
      ID 213212: 0.2088
      ID 82929: 0.1897
      ID 12461: 0.1877
      ID 101184: 0.1864
      ID 125510: 0.1702

🔸 多向量嵌入 (ColBERT Embedding)
  - 向量数量: 26
  - 每个向量维度: 1024
  - 向量间平均余弦相似度: 0.6706
  - 第1个向量前10维: [-0.05067746  0.00912721 -0.0287433  -0.0469315  -0.01172367  0.00606936
  0.00621777 -0.0672

## 多模态嵌入

In [12]:
"""
多模态嵌入检索系统：使用 Visualized-BGE 模型实现图文联合检索

Visualized-BGE 是一个强大的多模态嵌入模型，支持：
- 纯图片编码
- 纯文本编码
- 图文联合编码
- 跨模态检索（以图搜文、以文搜图、以图搜图）

安装说明：
1. 安装 visual_bge：https://github.com/FlagOpen/FlagEmbedding/tree/master/research/visual_bge
2. 下载模型权重：wget https://huggingface.co/BAAI/bge-visualized/resolve/main/Visualized_base_en_v1.5.pth

注意：如果虚拟环境无法使用，请切换到物理环境
"""

import torch
from visual_bge.modeling import Visualized_BGE
from PIL import Image
import numpy as np
from typing import List, Dict, Tuple, Union, Optional
import os
import time
from pathlib import Path


class MultimodalRetriever:
    """
    多模态检索器
    
    支持三种检索模式：
    1. Image-to-Image: 以图搜图
    2. Image-to-Text: 以图搜文
    3. Text-to-Image: 以文搜图
    
    参数:
        model_name_bge: BGE 文本模型名称
        model_weight: Visualized-BGE 模型权重路径
        device: 计算设备，默认自动选择
    """
    
    def __init__(
        self, 
        model_name_bge: str = "BAAI/bge-base-en-v1.5",
        model_weight: str = None,
        device: Optional[str] = None
    ):
        print(f"🔧 正在加载 Visualized-BGE 多模态模型...")
        start_time = time.time()
        
        # 自动选择设备
        if device is None:
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = device
        
        # 检查模型权重文件
        if model_weight is None or not os.path.exists(model_weight):
            raise FileNotFoundError(
                f"模型权重文件不存在: {model_weight}\n"
                "请下载: wget https://huggingface.co/BAAI/bge-visualized/resolve/main/Visualized_base_en_v1.5.pth"
            )
        
        # 初始化模型
        self.model = Visualized_BGE(
            model_name_bge=model_name_bge, 
            model_weight=model_weight
        )
        self.model.eval()
        
        # 存储索引数据
        self.image_embeddings = []
        self.text_embeddings = []
        self.image_paths = []
        self.texts = []
        
        load_time = time.time() - start_time
        print(f"✓ 模型加载完成 (设备: {self.device}, 耗时: {load_time:.2f}秒)")
    
    def encode_image(
        self, 
        image_path: Union[str, List[str]], 
        text: Optional[Union[str, List[str]]] = None
    ) -> np.ndarray:
        """
        编码图片（可选附加文本描述）
        
        参数:
            image_path: 图片路径或路径列表
            text: 可选的文本描述，用于图文联合编码
            
        返回:
            嵌入向量数组
        """
        # 处理单个或批量输入
        if isinstance(image_path, str):
            image_path = [image_path]
        if text is not None and isinstance(text, str):
            text = [text]
        
        embeddings = []
        with torch.no_grad():
            for idx, img_path in enumerate(image_path):
                if not os.path.exists(img_path):
                    print(f"⚠️  图片不存在，跳过: {img_path}")
                    continue
                
                # 编码图片（带或不带文本）
                img_text = text[idx] if text is not None else None
                embedding = self.model.encode(image=img_path, text=img_text)
                embeddings.append(embedding.cpu().numpy())
        
        return np.vstack(embeddings) if embeddings else np.array([])
    
    def encode_text(self, text: Union[str, List[str]]) -> np.ndarray:
        """
        编码纯文本
        
        参数:
            text: 文本或文本列表
            
        返回:
            嵌入向量数组
        """
        if isinstance(text, str):
            text = [text]
        
        embeddings = []
        with torch.no_grad():
            for t in text:
                embedding = self.model.encode(text=t)
                embeddings.append(embedding.cpu().numpy())
        
        return np.vstack(embeddings)
    
    def index_images(
        self, 
        image_paths: List[str], 
        descriptions: Optional[List[str]] = None
    ):
        """
        索引图片集合
        
        参数:
            image_paths: 图片路径列表
            descriptions: 可选的图片描述列表
        """
        print(f"\n{'='*80}")
        print(f"【步骤 1】索引图片集合")
        print(f"{'='*80}")
        print(f"图片数量: {len(image_paths)}")
        
        start_time = time.time()
        self.image_paths = image_paths
        self.image_embeddings = self.encode_image(image_paths, descriptions)
        
        index_time = time.time() - start_time
        print(f"✓ 索引完成 (耗时: {index_time:.2f}秒, 速度: {len(image_paths)/index_time:.2f} 图/秒)")
        print(f"  - 嵌入维度: {self.image_embeddings.shape[1]}")
    
    def index_texts(self, texts: List[str]):
        """
        索引文本集合
        
        参数:
            texts: 文本列表
        """
        print(f"\n{'='*80}")
        print(f"【步骤 1】索引文本集合")
        print(f"{'='*80}")
        print(f"文本数量: {len(texts)}")
        
        start_time = time.time()
        self.texts = texts
        self.text_embeddings = self.encode_text(texts)
        
        index_time = time.time() - start_time
        print(f"✓ 索引完成 (耗时: {index_time:.2f}秒)")
        print(f"  - 嵌入维度: {self.text_embeddings.shape[1]}")
    
    def search_by_image(
        self, 
        query_image: str, 
        search_in: str = 'images',
        top_k: int = 3
    ) -> List[Tuple[int, float, str]]:
        """
        以图搜索
        
        参数:
            query_image: 查询图片路径
            search_in: 搜索目标，'images' 或 'texts'
            top_k: 返回前 k 个结果
            
        返回:
            [(索引, 相似度分数, 内容), ...]
        """
        query_emb = self.encode_image(query_image)
        
        if search_in == 'images':
            if len(self.image_embeddings) == 0:
                raise ValueError("请先使用 index_images() 索引图片")
            scores = self._compute_similarities(query_emb, self.image_embeddings)
            results = [(i, scores[i], self.image_paths[i]) for i in range(len(scores))]
        else:  # texts
            if len(self.text_embeddings) == 0:
                raise ValueError("请先使用 index_texts() 索引文本")
            scores = self._compute_similarities(query_emb, self.text_embeddings)
            results = [(i, scores[i], self.texts[i]) for i in range(len(scores))]
        
        results.sort(key=lambda x: x[1], reverse=True)
        return results[:top_k]
    
    def search_by_text(
        self, 
        query_text: str,
        search_in: str = 'images',
        top_k: int = 3
    ) -> List[Tuple[int, float, str]]:
        """
        以文搜索
        
        参数:
            query_text: 查询文本
            search_in: 搜索目标，'images' 或 'texts'
            top_k: 返回前 k 个结果
            
        返回:
            [(索引, 相似度分数, 内容), ...]
        """
        query_emb = self.encode_text(query_text)
        
        if search_in == 'images':
            if len(self.image_embeddings) == 0:
                raise ValueError("请先使用 index_images() 索引图片")
            scores = self._compute_similarities(query_emb, self.image_embeddings)
            results = [(i, scores[i], self.image_paths[i]) for i in range(len(scores))]
        else:  # texts
            if len(self.text_embeddings) == 0:
                raise ValueError("请先使用 index_texts() 索引文本")
            scores = self._compute_similarities(query_emb, self.text_embeddings)
            results = [(i, scores[i], self.texts[i]) for i in range(len(scores))]
        
        results.sort(key=lambda x: x[1], reverse=True)
        return results[:top_k]
    
    def _compute_similarities(self, query_emb: np.ndarray, doc_embs: np.ndarray) -> np.ndarray:
        """
        计算余弦相似度
        
        参数:
            query_emb: 查询嵌入 (1, dim)
            doc_embs: 文档嵌入 (n, dim)
            
        返回:
            相似度数组 (n,)
        """
        # 归一化
        query_norm = query_emb / np.linalg.norm(query_emb, axis=1, keepdims=True)
        doc_norms = doc_embs / np.linalg.norm(doc_embs, axis=1, keepdims=True)
        
        # 余弦相似度
        similarities = np.dot(query_norm, doc_norms.T)[0]
        return similarities
    
    def compute_similarity(
        self, 
        item1: Union[str, np.ndarray],
        item2: Union[str, np.ndarray],
        item1_type: str = 'image',
        item2_type: str = 'image'
    ) -> float:
        """
        计算两个项目之间的相似度
        
        参数:
            item1: 第一个项目（路径或嵌入向量）
            item2: 第二个项目（路径或嵌入向量）
            item1_type: 'image' 或 'text'
            item2_type: 'image' 或 'text'
            
        返回:
            相似度分数 [0, 1]
        """
        # 获取嵌入向量
        if isinstance(item1, str):
            emb1 = self.encode_image(item1) if item1_type == 'image' else self.encode_text(item1)
        else:
            emb1 = item1
        
        if isinstance(item2, str):
            emb2 = self.encode_image(item2) if item2_type == 'image' else self.encode_text(item2)
        else:
            emb2 = item2
        
        # 计算余弦相似度
        similarity = np.dot(emb1.flatten(), emb2.flatten()) / (
            np.linalg.norm(emb1) * np.linalg.norm(emb2)
        )
        
        return float(similarity)
    
    def analyze_embedding(self, item: str, item_type: str = 'image', text_desc: Optional[str] = None):
        """
        分析嵌入向量的特性
        
        参数:
            item: 图片路径或文本
            item_type: 'image' 或 'text'
            text_desc: 如果是图片，可选的文本描述
        """
        print(f"\n{'='*80}")
        print(f"📊 嵌入向量分析")
        print(f"{'='*80}")
        
        if item_type == 'image':
            print(f"图片: {item}")
            if text_desc:
                print(f"描述: {text_desc}")
            
            # 纯图片嵌入
            img_emb = self.encode_image(item)
            print(f"\n🖼️  纯图片嵌入:")
            self._print_embedding_stats(img_emb[0])
            
            # 图文联合嵌入（如果有描述）
            if text_desc:
                multimodal_emb = self.encode_image(item, text_desc)
                print(f"\n🖼️ +📝 图文联合嵌入:")
                self._print_embedding_stats(multimodal_emb[0])
                
                # 计算两种嵌入的相似度
                similarity = self.compute_similarity(img_emb, multimodal_emb)
                print(f"\n相似度 (纯图片 vs 图文联合): {similarity:.4f}")
        else:
            print(f"文本: {item}")
            text_emb = self.encode_text(item)
            print(f"\n📝 文本嵌入:")
            self._print_embedding_stats(text_emb[0])
    
    def _print_embedding_stats(self, embedding: np.ndarray):
        """打印嵌入向量统计信息"""
        print(f"  - 维度: {embedding.shape[0]}")
        print(f"  - L2 范数: {np.linalg.norm(embedding):.4f}")
        print(f"  - 均值: {np.mean(embedding):.6f}")
        print(f"  - 标准差: {np.std(embedding):.6f}")
        print(f"  - 最小值: {np.min(embedding):.6f}")
        print(f"  - 最大值: {np.max(embedding):.6f}")
        print(f"  - 前10维: {embedding[:10]}")


def main():
    """主函数：演示多模态检索系统"""
    
    print("="*80)
    print("🎨 Visualized-BGE 多模态检索系统演示")
    print("="*80)
    
    # 配置路径
    base_dir = "/Users/ll/Desktop/rag/rag-in-action"
    model_weight = os.path.join(base_dir, "Visualized_base_en_v1.5.pth")
    data_dir = os.path.join(base_dir, "90-文档-Data/多模态")
    
    # 检查模型权重是否存在
    if not os.path.exists(model_weight):
        print(f"❌ 模型权重文件不存在: {model_weight}")
        print("请下载模型权重文件：")
        print("wget https://huggingface.co/BAAI/bge-visualized/resolve/main/Visualized_base_en_v1.5.pth")
        return
    
    # 检查数据目录
    if not os.path.exists(data_dir):
        print(f"❌ 数据目录不存在: {data_dir}")
        print("请创建数据目录并准备示例图片")
        return
    
    # 创建检索器
    retriever = MultimodalRetriever(
        model_name_bge="BAAI/bge-base-en-v1.5",
        model_weight=model_weight
    )
    
    # 示例图片路径
    query_image = os.path.join(data_dir, "query_image.jpg")
    
    if not os.path.exists(query_image):
        print(f"❌ 查询图片不存在: {query_image}")
        print("请准备示例图片")
        return
    
    # 【演示 1】嵌入向量分析
    print(f"\n{'='*80}")
    print("【演示 1】嵌入向量特性分析")
    print(f"{'='*80}")
    
    retriever.analyze_embedding(
        query_image, 
        item_type='image',
        text_desc="这是一张悟空战斗的示例图片"
    )
    
    # 【演示 2】文本嵌入分析
    retriever.analyze_embedding(
        "A powerful warrior fighting monsters with flame fist",
        item_type='text'
    )
    
    # 【演示 3】跨模态相似度计算
    print(f"\n{'='*80}")
    print("【演示 2】跨模态相似度计算")
    print(f"{'='*80}")
    
    text_queries = [
        "悟空战斗场景",
        "A warrior with flame abilities",
        "Cat playing with ball",
        "Beautiful landscape scenery"
    ]
    
    print(f"\n查询图片: {os.path.basename(query_image)}")
    print("-" * 80)
    
    for text in text_queries:
        similarity = retriever.compute_similarity(
            query_image, text,
            item1_type='image', item2_type='text'
        )
        print(f"  与文本 '{text}' 的相似度: {similarity:.4f}")
    
    # 【演示 3】如果有多张图片，演示检索功能
    print(f"\n{'='*80}")
    print("【演示 3】图片检索功能示例")
    print(f"{'='*80}")
    
    # 查找数据目录下的所有图片
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif'}
    image_files = [
        os.path.join(data_dir, f) 
        for f in os.listdir(data_dir)
        if os.path.splitext(f)[1].lower() in image_extensions
    ]
    
    if len(image_files) > 1:
        # 图片描述（可选）
        descriptions = [f"Image {i+1}" for i in range(len(image_files))]
        
        # 索引图片
        retriever.index_images(image_files, descriptions)
        
        # 以图搜图
        print(f"\n🔍 以图搜图:")
        print(f"查询图片: {os.path.basename(query_image)}")
        print("-" * 80)
        
        results = retriever.search_by_image(query_image, search_in='images', top_k=3)
        for rank, (idx, score, img_path) in enumerate(results, 1):
            print(f"  {rank}. 相似度: {score:.4f} - {os.path.basename(img_path)}")
        
        # 以文搜图
        print(f"\n🔍 以文搜图:")
        search_text = "warrior fighting scene"
        print(f"查询文本: {search_text}")
        print("-" * 80)
        
        results = retriever.search_by_text(search_text, search_in='images', top_k=3)
        for rank, (idx, score, img_path) in enumerate(results, 1):
            print(f"  {rank}. 相似度: {score:.4f} - {os.path.basename(img_path)}")
    else:
        print("⚠️  数据目录中图片数量不足，跳过检索演示")
        print(f"当前图片数量: {len(image_files)}")
    
    # 总结
    print(f"\n{'='*80}")
    print("【多模态检索应用场景】")
    print(f"{'='*80}")
    print("""
🎯 应用场景:
  1. 以图搜图: 图片去重、相似图片推荐、版权检测
  2. 以文搜图: 图片搜索引擎、电商商品检索、设计素材查找
  3. 以图搜文: 图片标注、内容审核、图片理解
  4. 图文匹配: 广告投放、内容推荐、多模态问答

💡 技术优势:
  - 统一的嵌入空间: 图片和文本在同一向量空间中表示
  - 跨模态检索: 支持图找文、文找图等灵活检索方式
  - 图文增强: 结合图片和文本描述，提升表示质量
  - 高效检索: 向量化表示支持快速相似度计算
    """)
    
    print(f"\n{'='*80}")
    print("✓ 演示完成！")
    print(f"{'='*80}")


if __name__ == '__main__':
    main() 

🎨 Visualized-BGE 多模态检索系统演示
🔧 正在加载 Visualized-BGE 多模态模型...
✓ 模型加载完成 (设备: cpu, 耗时: 8.62秒)

【演示 1】嵌入向量特性分析

📊 嵌入向量分析
图片: /Users/ll/Desktop/rag/rag-in-action/90-文档-Data/多模态/query_image.jpg
描述: 这是一张悟空战斗的示例图片

🖼️  纯图片嵌入:
  - 维度: 768
  - L2 范数: 1.0000
  - 均值: -0.000651
  - 标准差: 0.036079
  - 最小值: -0.250460
  - 最大值: 0.096528
  - 前10维: [ 0.01775778 -0.02236971  0.004773    0.00458726  0.02112675 -0.00138004
  0.07065601  0.0665949  -0.01240909 -0.01269231]

🖼️ +📝 图文联合嵌入:
  - 维度: 768
  - L2 范数: 1.0000
  - 均值: -0.000667
  - 标准差: 0.036078
  - 最小值: -0.250252
  - 最大值: 0.098347
  - 前10维: [ 0.02958835 -0.02625961  0.00796746  0.00045585  0.00366952 -0.00388724
  0.05554389  0.0655842  -0.01739006 -0.02965876]

相似度 (纯图片 vs 图文联合): 0.8931

📊 嵌入向量分析
文本: A powerful warrior fighting monsters with flame fist

📝 文本嵌入:
  - 维度: 768
  - L2 范数: 1.0000
  - 均值: -0.000663
  - 标准差: 0.036078
  - 最小值: -0.242385
  - 最大值: 0.104005
  - 前10维: [-4.0013447e-02  7.9561156e-05 -4.8375305e-02  1.4259506e-02
 -1.1663715e-02 -1.86

# Milvus向量数据库

In [9]:
"""
Milvus 向量数据库完整操作指南

本文件整合了 Milvus 的核心功能演示：
1. 数据库管理 (Database Management)
2. 集合管理 (Collection Management)
3. Schema 定义 (Schema Definition)
4. 实体操作 (Entity Operations)

安装依赖：
    pip install pymilvus

启动 Milvus 服务端：
    wget https://github.com/milvus-io/milvus/releases/download/v2.5.10/milvus-standalone-docker-compose.yml -O docker-compose.yml
    sudo docker compose up -d

检查版本：
    pip show pymilvus
"""

from pymilvus import MilvusClient, DataType
from pymilvus.exceptions import MilvusException
import random
from typing import List, Dict, Any, Optional
import time


class MilvusManager:
    """
    Milvus 向量数据库管理器
    
    封装了 Milvus 的完整操作流程：
    - 数据库 CRUD
    - 集合 CRUD
    - Schema 定义
    - 实体 CRUD
    - 分区管理
    - 别名管理
    
    参数:
        uri: Milvus 服务地址，默认 "http://localhost:19530"
        token: 认证令牌，格式 "用户名:密码"，默认 "root:Milvus"
    """
    
    def __init__(self, uri: str = "http://localhost:19530", token: str = "root:Milvus"):
        print(f"🔧 正在连接 Milvus...")
        self.client = MilvusClient(uri=uri, token=token)
        print(f"✓ 已连接到 Milvus (URI: {uri})")
    
    # ========================================
    # 第一部分：数据库管理
    # ========================================
    
    def demo_database_operations(self):
        """演示数据库的完整操作"""
        print(f"\n{'='*80}")
        print("【第一部分】数据库管理 (Database Management)")
        print(f"{'='*80}")
        
        # 1. 创建数据库
        print(f"\n1️⃣  创建数据库")
        print("-" * 80)
        
        # 创建基本数据库
        try:
            self.client.create_database(db_name="demo_db_1")
            print("✓ demo_db_1 创建成功")
        except MilvusException as e:
            if "already exist" in str(e):
                print("ℹ demo_db_1 已存在，跳过创建")
            else:
                raise
        
        # 创建带属性的数据库（设置副本数）
        try:
            self.client.create_database(
                db_name="demo_db_2",
                properties={"database.replica.number": 2}
            )
            print("✓ demo_db_2 创建成功 (副本数=2)")
        except MilvusException as e:
            if "already exist" in str(e):
                print("ℹ demo_db_2 已存在，跳过创建")
            else:
                raise
        
        # 2. 列出所有数据库
        print(f"\n2️⃣  列出所有数据库")
        print("-" * 80)
        db_list = self.client.list_databases()
        print(f"当前数据库列表: {db_list}")
        
        # 3. 查看数据库详情
        print(f"\n3️⃣  查看数据库详情")
        print("-" * 80)
        default_info = self.client.describe_database(db_name="default")
        print(f"默认数据库详情: {default_info}")
        
        # 4. 修改数据库属性
        print(f"\n4️⃣  修改数据库属性")
        print("-" * 80)
        self.client.alter_database_properties(
            db_name="demo_db_1",
            properties={"database.max.collections": 10}
        )
        print("✓ 已为 demo_db_1 限制最大集合数为 10")
        
        # 5. 删除数据库属性
        print(f"\n5️⃣  删除数据库属性")
        print("-" * 80)
        self.client.drop_database_properties(
            db_name="demo_db_1",
            property_keys=["database.max.collections"]
        )
        print("✓ 已移除 demo_db_1 的最大集合数限制")
        
        # 6. 切换数据库
        print(f"\n6️⃣  切换当前数据库")
        print("-" * 80)
        self.client.use_database(db_name="demo_db_1")
        print("✓ 已切换到 demo_db_1")
        
        # 切换回默认数据库
        self.client.use_database(db_name="default")
        print("✓ 已切换回 default 数据库")
    
    def cleanup_databases(self):
        """清理演示数据库"""
        print(f"\n🧹 清理演示数据库...")
        for db_name in ["demo_db_1", "demo_db_2"]:
            try:
                self.client.drop_database(db_name=db_name)
                print(f"✓ 已删除 {db_name}")
            except Exception as e:
                print(f"⚠️  删除 {db_name} 失败: {e}")
    
    # ========================================
    # 第二部分：集合管理
    # ========================================
    
    def demo_collection_operations(self):
        """演示集合的完整操作"""
        print(f"\n{'='*80}")
        print("【第二部分】集合管理 (Collection Management)")
        print(f"{'='*80}")
        
        collection_name = "demo_collection"
        
        # 1. 创建集合（快速模式）
        print(f"\n1️⃣  创建集合 (快速模式)")
        print("-" * 80)
        
        if collection_name in self.client.list_collections():
            self.client.drop_collection(collection_name=collection_name)
            print(f"✓ 已删除已存在的集合 {collection_name}")
        
        self.client.create_collection(
            collection_name=collection_name,
            dimension=128  # 向量维度
        )
        print(f"✓ 集合 {collection_name} 创建成功 (维度=128)")
        
        # 2. 列出所有集合
        print(f"\n2️⃣  列出所有集合")
        print("-" * 80)
        collections = self.client.list_collections()
        print(f"当前集合列表: {collections}")
        
        # 3. 查看集合详情
        print(f"\n3️⃣  查看集合详情")
        print("-" * 80)
        info = self.client.describe_collection(collection_name=collection_name)
        print(f"集合详情:")
        print(f"  - 集合名: {info.get('collection_name')}")
        print(f"  - 字段数: {len(info.get('fields', []))}")
        
        # 4. 重命名集合
        print(f"\n4️⃣  重命名集合")
        print("-" * 80)
        new_name = "demo_collection_renamed"
        
        if new_name in self.client.list_collections():
            self.client.drop_collection(collection_name=new_name)
        
        self.client.rename_collection(
            old_name=collection_name,
            new_name=new_name
        )
        print(f"✓ 集合已重命名: {collection_name} -> {new_name}")
        collection_name = new_name
        
        # 5. 修改集合属性（设置 TTL）
        print(f"\n5️⃣  修改集合属性")
        print("-" * 80)
        self.client.alter_collection_properties(
            collection_name=collection_name,
            properties={"collection.ttl.seconds": 3600}
        )
        print(f"✓ 已为 {collection_name} 设置 TTL=3600秒 (1小时)")
        
        # 6. 删除集合属性
        print(f"\n6️⃣  删除集合属性")
        print("-" * 80)
        self.client.drop_collection_properties(
            collection_name=collection_name,
            property_keys=["collection.ttl.seconds"]
        )
        print(f"✓ 已删除 {collection_name} 的 TTL 属性")
        
        # 7. 加载和释放集合
        print(f"\n7️⃣  加载和释放集合")
        print("-" * 80)
        
        self.client.load_collection(collection_name=collection_name)
        state = self.client.get_load_state(collection_name=collection_name)
        print(f"加载状态: {state}")
        
        self.client.release_collection(collection_name=collection_name)
        state = self.client.get_load_state(collection_name=collection_name)
        print(f"释放后状态: {state}")
        
        # 8. 分区管理
        print(f"\n8️⃣  分区管理 (Partition Management)")
        print("-" * 80)
        
        # 列出分区
        partitions = self.client.list_partitions(collection_name=collection_name)
        print(f"初始分区列表: {partitions}")
        
        # 创建分区
        partition_name = "demo_partition"
        self.client.create_partition(
            collection_name=collection_name,
            partition_name=partition_name
        )
        print(f"✓ 已创建分区 {partition_name}")
        
        # 检查分区是否存在
        exists = self.client.has_partition(
            collection_name=collection_name,
            partition_name=partition_name
        )
        print(f"分区 {partition_name} 存在: {exists}")
        
        # 加载分区
        self.client.load_partitions(
            collection_name=collection_name,
            partition_names=[partition_name]
        )
        print(f"✓ 已加载分区 {partition_name}")
        
        # 释放分区
        self.client.release_partitions(
            collection_name=collection_name,
            partition_names=[partition_name]
        )
        print(f"✓ 已释放分区 {partition_name}")
        
        # 删除分区
        self.client.drop_partition(
            collection_name=collection_name,
            partition_name=partition_name
        )
        print(f"✓ 已删除分区 {partition_name}")
        
        # 9. 别名管理
        print(f"\n9️⃣  别名管理 (Alias Management)")
        print("-" * 80)
        
        # 创建别名
        alias1 = "demo_alias_1"
        alias2 = "demo_alias_2"
        
        self.client.create_alias(collection_name=collection_name, alias=alias1)
        self.client.create_alias(collection_name=collection_name, alias=alias2)
        print(f"✓ 已创建别名: {alias1}, {alias2}")
        
        # 列出别名
        aliases = self.client.list_aliases(collection_name=collection_name)
        print(f"当前别名列表: {aliases}")
        
        # 查看别名详情
        desc = self.client.describe_alias(alias=alias1)
        print(f"别名 {alias1} 详情: {desc}")
        
        # 删除别名
        self.client.drop_alias(alias=alias1)
        print(f"✓ 已删除别名 {alias1}")
        
        # 清理
        self.client.drop_alias(alias=alias2)
        self.client.drop_collection(collection_name=collection_name)
        print(f"\n✓ 已清理演示集合")
    
    # ========================================
    # 第三部分：Schema 定义
    # ========================================
    
    def demo_schema_definition(self):
        """演示 Schema 的定义和使用"""
        print(f"\n{'='*80}")
        print("【第三部分】Schema 定义 (Schema Definition)")
        print(f"{'='*80}")
        
        # 1. 创建空 Schema
        print(f"\n1️⃣  创建 Schema")
        print("-" * 80)
        schema = MilvusClient.create_schema()
        print("✓ 已创建空 Schema")
        
        # 2. 添加主键字段
        print(f"\n2️⃣  添加主键字段 (Primary Key)")
        print("-" * 80)
        
        # INT64 主键
        schema.add_field(
            field_name="id",
            datatype=DataType.INT64,
            is_primary=True,  # 设置为主键
            auto_id=False     # 手动指定 ID
        )
        print("✓ 已添加 INT64 类型主键字段 'id'")
        
        # 3. 添加向量字段
        print(f"\n3️⃣  添加向量字段 (Vector Fields)")
        print("-" * 80)
        
        # 密集浮点向量
        schema.add_field(
            field_name="text_embedding",
            datatype=DataType.FLOAT_VECTOR,
            dim=768  # 向量维度
        )
        print("✓ 已添加 FLOAT_VECTOR 字段 'text_embedding' (维度=768)")
        
        # 二进制向量
        schema.add_field(
            field_name="image_embedding",
            datatype=DataType.BINARY_VECTOR,
            dim=256  # 维度必须是 8 的倍数
        )
        print("✓ 已添加 BINARY_VECTOR 字段 'image_embedding' (维度=256)")
        
        # 4. 添加标量字段
        print(f"\n4️⃣  添加标量字段 (Scalar Fields)")
        print("-" * 80)
        
        # 字符串字段
        schema.add_field(
            field_name="title",
            datatype=DataType.VARCHAR,
            max_length=200,
            is_nullable=True,
            default_value="untitled"
        )
        print("✓ 已添加 VARCHAR 字段 'title' (最大长度=200)")
        
        # 整数字段
        schema.add_field(
            field_name="score",
            datatype=DataType.INT32,
            is_nullable=False
        )
        print("✓ 已添加 INT32 字段 'score'")
        
        # 双精度浮点数字段
        schema.add_field(
            field_name="price",
            datatype=DataType.DOUBLE,
            default_value=0.0
        )
        print("✓ 已添加 DOUBLE 字段 'price'")
        
        # 布尔字段
        schema.add_field(
            field_name="is_active",
            datatype=DataType.BOOL,
            default_value=True
        )
        print("✓ 已添加 BOOL 字段 'is_active'")
        
        # JSON 字段
        schema.add_field(
            field_name="metadata",
            datatype=DataType.JSON
        )
        print("✓ 已添加 JSON 字段 'metadata'")
        
        # 数组字段
        schema.add_field(
            field_name="tags",
            datatype=DataType.ARRAY,
            element_type=DataType.VARCHAR,
            max_capacity=10,
            max_length=50
        )
        print("✓ 已添加 ARRAY 字段 'tags' (容量=10)")
        
        # 5. 使用 Schema 创建集合
        print(f"\n5️⃣  使用 Schema 创建集合")
        print("-" * 80)
        
        collection_name = "demo_schema_collection"
        
        if collection_name in self.client.list_collections():
            self.client.drop_collection(collection_name=collection_name)
        
        self.client.create_collection(
            collection_name=collection_name,
            schema=schema
        )
        print(f"✓ 已使用自定义 Schema 创建集合 '{collection_name}'")
        
        # 6. 查看集合详情
        print(f"\n6️⃣  查看集合详情")
        print("-" * 80)
        info = self.client.describe_collection(collection_name=collection_name)
        print(f"集合 '{collection_name}' 的字段:")
        for field in info.get('fields', []):
            print(f"  - {field.get('name')}: {field.get('type')} "
                  f"{'(主键)' if field.get('is_primary') else ''}")
        
        # 清理
        self.client.drop_collection(collection_name=collection_name)
        print(f"\n✓ 已清理演示集合")
    
    # ========================================
    # 第四部分：实体操作
    # ========================================
    
    def demo_entity_operations(self):
        """演示实体的 CRUD 操作"""
        print(f"\n{'='*80}")
        print("【第四部分】实体操作 (Entity Operations)")
        print(f"{'='*80}")
        
        collection_name = "demo_entity_collection"
        
        # 1. 创建集合
        print(f"\n1️⃣  创建集合")
        print("-" * 80)
        
        if collection_name in self.client.list_collections():
            self.client.drop_collection(collection_name=collection_name)
        
        self.client.create_collection(
            collection_name=collection_name,
            dimension=5,  # 向量维度
            primary_field_name="id",
            vector_field_name="vector",
            id_type="int"
        )
        print(f"✓ 集合 '{collection_name}' 创建成功 (维度=5)")
        
        # 2. 插入实体
        print(f"\n2️⃣  插入实体 (Insert)")
        print("-" * 80)
        
        # 生成示例数据
        data = [
            {
                "id": i,
                "vector": [random.random() for _ in range(5)],
                "color": f"color_{i}"
            }
            for i in range(10)
        ]
        
        res = self.client.insert(
            collection_name=collection_name,
            data=data
        )
        print(f"✓ 已插入 {res['insert_count']} 条实体")
        print(f"  插入的 ID 范围: {res['ids'][:3]}...{res['ids'][-1]}")
        
        # 3. 更新实体（Upsert）
        print(f"\n3️⃣  更新实体 (Upsert)")
        print("-" * 80)
        
        update_data = [
            {
                "id": 0,
                "vector": [random.random() for _ in range(5)],
                "color": "updated_color_0"
            },
            {
                "id": 1,
                "vector": [random.random() for _ in range(5)],
                "color": "updated_color_1"
            }
        ]
        
        res = self.client.upsert(
            collection_name=collection_name,
            data=update_data
        )
        print(f"✓ 已更新 {res['upsert_count']} 条实体 (ID: {res['ids']})")
        
        # 4. 刷新和加载
        print(f"\n4️⃣  刷新和加载集合")
        print("-" * 80)
        
        self.client.flush(collection_name=collection_name)
        print("✓ 已刷新集合（数据持久化到磁盘）")
        
        self.client.load_collection(collection_name=collection_name)
        print("✓ 已加载集合到内存")
        
        # 5. 查询实体
        print(f"\n5️⃣  查询实体 (Query)")
        print("-" * 80)
        
        # 按 ID 查询
        res = self.client.query(
            collection_name=collection_name,
            filter="id in [0, 1, 2]",
            output_fields=["id", "color"]
        )
        print(f"查询结果 (ID in [0,1,2]):")
        for entity in res:
            print(f"  - ID: {entity['id']}, Color: {entity['color']}")
        
        # 6. 向量检索
        print(f"\n6️⃣  向量检索 (Vector Search)")
        print("-" * 80)
        
        # 随机查询向量
        query_vector = [random.random() for _ in range(5)]
        
        search_res = self.client.search(
            collection_name=collection_name,
            data=[query_vector],
            limit=3,
            output_fields=["id", "color"]
        )
        
        print(f"Top-3 最相似的实体:")
        for hits in search_res:
            for rank, hit in enumerate(hits, 1):
                print(f"  {rank}. ID: {hit['id']}, "
                      f"Color: {hit['entity']['color']}, "
                      f"距离: {hit['distance']:.4f}")
        
        # 7. 混合检索（向量 + 过滤）
        print(f"\n7️⃣  混合检索 (Hybrid Search)")
        print("-" * 80)
        
        search_res = self.client.search(
            collection_name=collection_name,
            data=[query_vector],
            filter="id > 5",  # 过滤条件
            limit=3,
            output_fields=["id", "color"]
        )
        
        print(f"Top-3 最相似的实体 (ID > 5):")
        for hits in search_res:
            for rank, hit in enumerate(hits, 1):
                print(f"  {rank}. ID: {hit['id']}, "
                      f"Color: {hit['entity']['color']}, "
                      f"距离: {hit['distance']:.4f}")
        
        # 8. 删除实体
        print(f"\n8️⃣  删除实体 (Delete)")
        print("-" * 80)
        
        res = self.client.delete(
            collection_name=collection_name,
            ids=[0, 1]
        )
        print(f"✓ 已删除实体 (ID: [0, 1])")
        
        # 验证删除
        res = self.client.query(
            collection_name=collection_name,
            filter="id in [0, 1, 2]",
            output_fields=["id", "color"]
        )
        print(f"验证删除后查询结果: {[e['id'] for e in res]}")
        
        # 9. 统计信息
        print(f"\n9️⃣  集合统计信息")
        print("-" * 80)
        
        stats = self.client.get_collection_stats(collection_name=collection_name)
        print(f"集合统计:")
        print(f"  - 总实体数: {stats.get('row_count', 0)}")
        
        # 清理
        self.client.drop_collection(collection_name=collection_name)
        print(f"\n✓ 已清理演示集合")
    
    # ========================================
    # 综合演示
    # ========================================
    
    def cleanup_before_demo(self):
        """演示前清理，确保环境干净"""
        print(f"\n🧹 清理旧的演示数据...")
        
        # 清理旧的数据库
        for db_name in ["demo_db_1", "demo_db_2"]:
            try:
                if db_name in self.client.list_databases():
                    self.client.drop_database(db_name=db_name)
                    print(f"  ✓ 已删除旧数据库 {db_name}")
            except Exception as e:
                print(f"  ⚠️  清理 {db_name} 时出错: {e}")
        
        print("✓ 清理完成\n")
    
    def run_complete_demo(self):
        """运行完整演示"""
        print("="*80)
        print("🚀 Milvus 向量数据库完整操作演示")
        print("="*80)
        
        try:
            # 演示前清理
            self.cleanup_before_demo()
            # 第一部分：数据库管理
            self.demo_database_operations()
            
            # 第二部分：集合管理
            self.demo_collection_operations()
            
            # 第三部分：Schema 定义
            self.demo_schema_definition()
            
            # 第四部分：实体操作
            self.demo_entity_operations()
            
            # 清理演示数据库
            self.cleanup_databases()
            
            # 总结
            print(f"\n{'='*80}")
            print("【Milvus 核心概念总结】")
            print(f"{'='*80}")
            print("""
📚 核心概念层级:
  Database (数据库)
    └── Collection (集合)
          ├── Schema (模式定义)
          │     ├── Primary Field (主键字段)
          │     ├── Vector Field (向量字段)
          │     └── Scalar Field (标量字段)
          ├── Partition (分区)
          ├── Alias (别名)
          └── Entity (实体/数据)

🔑 关键操作:
  1. Database: 多租户隔离、资源管理
  2. Collection: 数据容器、Schema 定义
  3. Schema: 字段类型、约束条件
  4. Entity: CRUD 操作、向量检索

💡 最佳实践:
  - 合理规划 Database 和 Collection 结构
  - 根据业务需求设计 Schema
  - 使用 Partition 优化大规模数据查询
  - 利用 Alias 实现灰度发布和版本管理
  - 定期刷新(flush)保证数据持久化
  - 查询前确保 Collection 已加载到内存
            """)
            
            print(f"\n{'='*80}")
            print("✓ 所有演示完成！")
            print(f"{'='*80}")
            
        except Exception as e:
            print(f"\n❌ 演示过程中出错: {e}")
            import traceback
            traceback.print_exc()


def main():
    """主函数"""
    
    # 创建 Milvus 管理器
    try:
        manager = MilvusManager(
            uri="http://localhost:19530",
            token="root:Milvus"
        )
        
        # 运行完整演示
        manager.run_complete_demo()
        
    except Exception as e:
        print(f"\n❌ 连接 Milvus 失败: {e}")
        print("\n请确保:")
        print("  1. Milvus 服务已启动")
        print("  2. 端口 19530 可访问")
        print("  3. 认证信息正确")
        print("\n启动命令:")
        print("  wget https://github.com/milvus-io/milvus/releases/download/v2.5.10/milvus-standalone-docker-compose.yml -O docker-compose.yml")
        print("  sudo docker compose up -d")


if __name__ == '__main__':
    main()



🔧 正在连接 Milvus...
✓ 已连接到 Milvus (URI: http://localhost:19530)
🚀 Milvus 向量数据库完整操作演示

🧹 清理旧的演示数据...
✓ 清理完成


【第一部分】数据库管理 (Database Management)

1️⃣  创建数据库
--------------------------------------------------------------------------------
✓ demo_db_1 创建成功
✓ demo_db_2 创建成功 (副本数=2)

2️⃣  列出所有数据库
--------------------------------------------------------------------------------
当前数据库列表: ['default', 'demo_db_1', 'demo_db_2']

3️⃣  查看数据库详情
--------------------------------------------------------------------------------
默认数据库详情: {'name': 'default'}

4️⃣  修改数据库属性
--------------------------------------------------------------------------------
✓ 已为 demo_db_1 限制最大集合数为 10

5️⃣  删除数据库属性
--------------------------------------------------------------------------------
✓ 已移除 demo_db_1 的最大集合数限制

6️⃣  切换当前数据库
--------------------------------------------------------------------------------
✓ 已切换到 demo_db_1
✓ 已切换回 default 数据库

【第二部分】集合管理 (Collection Management)

1️⃣  创建集合 (快速模式)
--------------------------------

# 索引优化

## 从小块到上大上下文

### 节点句子华东窗口

In [20]:
# 临时绕过 torch.load 安全检查（仅用于开发环境）
# 注意：这是一个临时解决方案，生产环境应该升级 torch 到 2.6+ 或使用 safetensors 格式
import transformers.utils.import_utils
import transformers.modeling_utils

def dummy_check():
    pass

transformers.utils.import_utils.check_torch_load_is_safe = dummy_check
transformers.modeling_utils.check_torch_load_is_safe = dummy_check

from llama_index.core import  VectorStoreIndex, Settings, Document
from llama_index.core.node_parser import  SentenceWindowNodeParser, SentenceSplitter
from llama_index.llms.deepseek import DeepSeek
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.postprocessor import MetadataReplacementPostProcessor # 元数据替换后处理器

import os
os.environ["DEEPSEEK_API_KEY"] = "sk-34*****5bb4f"

# 配置全局设置
Settings.llm = DeepSeek(model="deepseek-chat", temperature=0.1)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-zh")
Settings.text_splitter = SentenceSplitter(separator="\n",  chunk_size=50, chunk_overlap=0)
# 准备知识文本并创建Document对象
game_knowledge = """
《灭神纪∙猢狲》是一款动作角色扮演游戏。游戏背景设定在架空的神话世界中。
玩家将扮演齐天大圣孙悟空，在充满东方神话元素的世界中展开冒险。
游戏的战斗系统极具特色，采用了独特的“变身系统”。悟空可以在战斗中变换不同形态。
每种形态都有其独特的战斗风格和技能组合。金刚形态侧重力量型打击，带来压倒性的破坏力。
魔佛形态则专注法术攻击，能释放强大的法术伤害。
游戏世界中充满了标志性的神话角色，除了主角孙悟空以外，还有来自佛教、道教等各派系的神魔。
这些角色既可能是悟空的盟友，也可能是需要击败的强大对手。
装备系统包含了丰富的武器选择，除了著名的如意金箍棒以外，悟空还可以使用各种神器法宝。
不同武器有其特色效果，玩家需要根据战斗场景灵活选择。
游戏的画面表现极具东方美学特色，场景融合了水墨画风格，将山川、建筑等元素完美呈现。
战斗特效既有中国传统文化元素，又具备现代游戏的视觉震撼力。
难度设计上，Boss战充满挑战性，需要玩家精准把握战斗节奏和技能运用。
同时游戏也提供了多种难度选择，照顾不同技术水平的玩家。"""
# 创建Document对象
documents = [Document(text=game_knowledge)]
# 创建带上下文窗口的句子解析器（每个目标句子两侧各保留n个句子作为上下文）
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text"
)
# 使用窗口解析器处理文档
nodes = node_parser.get_nodes_from_documents(documents)
# 使用基础解析器处理文档（用于对比）
base_nodes = Settings.text_splitter.get_nodes_from_documents(documents)
# 构建两种索引用于对比
sentence_index = VectorStoreIndex(nodes)
base_index = VectorStoreIndex(base_nodes)
# 创建带上下文窗口的查询引擎
window_query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ]
)
# 创建基础查询引擎
base_query_engine = base_index.as_query_engine(
    similarity_top_k=6
)
# 测试问答
test_questions = [
    "游戏中悟空有哪些形态变化？",
    # "游戏的画面风格是怎样的？",
    # "游戏的难度设计如何？"
]
print("=== 使用窗口解析器的检索结果 ===")
for question in test_questions:
    print(f"\n问题：{question}")
    window_response = window_query_engine.query(question)
    print(f"回答：{window_response}")
    
    # 展示检索到的原始句子和窗口内容
    print("\n检索详情：")
    for node in window_response.source_nodes:
        print(f"原始句子：{node.node.metadata['original_text']}")
        print(f"上下文窗口：{node.node.metadata['window']}")
        print("---")
print("\n=== 使用基础解析器的检索结果（对比）===")
for question in test_questions:
    print(f"\n问题：{question}")
    base_response = base_query_engine.query(question)
print(f"回答：{base_response}")


=== 使用窗口解析器的检索结果 ===

问题：游戏中悟空有哪些形态变化？


AuthenticationError: Error code: 401 - {'error': {'message': 'Authentication Fails, Your api key: ****bb4f is invalid', 'type': 'authentication_error', 'param': None, 'code': 'invalid_request_error'}}

### 父子文本块检索

In [ ]:
# 临时绕过 torch.load 安全检查（仅用于开发环境）
import transformers.utils.import_utils
import transformers.modeling_utils

def dummy_check():
    pass

transformers.utils.import_utils.check_torch_load_is_safe = dummy_check
transformers.modeling_utils.check_torch_load_is_safe = dummy_check

from langchain_deepseek import ChatDeepSeek 
from langchain_huggingface import HuggingFaceEmbeddings 

import os
os.environ["DEEPSEEK_API_KEY"] = "sk-34b4f11***5bb4f"

# 初始化语言模型和向量嵌入模型
llm = ChatDeepSeek(model="deepseek-chat", temperature=0.1)
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-zh")
# 准备游戏知识文本，创建Document对象。
from langchain_core.documents import Document
game_knowledge = """
《灭神纪∙猢狲》是一款动作角色扮演游戏。游戏背景设定在架空的神话世界中。玩家将扮演齐天大圣孙悟空，在充满东方神话元素的世界中展开冒险。游戏的战斗系统极具特色，采用了独特的"变身系统"。悟空可以在战斗中变换不同形态。每种形态都有其独特的战斗风格和技能组合。金刚形态侧重力量型打击，带来压倒性的破坏力。魔佛形态则专注法术攻击，能释放强大的法术伤害。游戏世界中充满了标志性的神话角色，除了主角孙悟空以外，还有来自佛教、道教等各派系的神魔。这些角色既可能是悟空的盟友，也可能是需要击败的强大对手。装备系统包含了丰富的武器选择，除了著名的如意金箍棒以外，悟空还可以使用各种神器法宝。不同武器有其特色效果，玩家需要根据战斗场景灵活选择。游戏的画面表现极具东方美学特色，场景融合了水墨画风格，将山川、建筑等元素完美呈现。战斗特效既有中国传统文化元素，又具备现代游戏的视觉震撼力。难度设计上，Boss战充满挑战性，需要玩家精准把握战斗节奏和技能运用。同时游戏也提供了多种难度选择，照顾不同技术水平的玩家。

"""
# 创建Document对象
documents = [Document(page_content=game_knowledge)]
from langchain_text_splitters import RecursiveCharacterTextSplitter
# 父文本块分割器（较大的文本块）
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", "。", "！", "？", "；", ",", " ", ""]
)
# 子文本块分割器（较小的文本块）
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
    separators=["\n\n", "\n", "。", "！", "？", "；", ",", " ", ""]
)
# 创建父子文本块
parent_docs = parent_splitter.split_documents(documents)
child_docs = child_splitter.split_documents(documents)
# 创建存储和检索器，建立两层存储系统
from langchain_classic.retrievers.parent_document_retriever import ParentDocumentRetriever # 父文档检索器
from langchain_core.stores import InMemoryStore # 内存存储
from langchain_community.vectorstores import Chroma # 向量存储
vectorstore = Chroma(
    collection_name="game_knowledge",
    embedding_function=embed_model
)
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore, # 向量存储
    docstore=store, # 文档存储
    child_splitter=child_splitter, # 子文本块分割器
    parent_splitter=parent_splitter, # 父文本块分割器
)
# 添加文本块
retriever.add_documents(documents)
# 自定义提示模板
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains.retrieval_qa.base import RetrievalQA
prompt_template = """基于以下上下文信息回答问题。如果无法找到答案，请说“我找不到相关信息”。
上下文：
{context}
问题：{question}
回答："""
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)
# 创建问答链
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # 问答链类型
    retriever=retriever,# 检索器
    return_source_documents=True, # 是否返回源文档
    chain_type_kwargs={"prompt": PROMPT}
)
# 通过实际问答测试系统
test_questions = [
    "游戏中悟空有哪些形态变化？",
    "游戏的画面风格是怎样的？",
]
for question in test_questions:
    print(f"\n问题：{question}")
    result = qa_chain({"query": question})    
    print(f"\n回答：{result['result']}")
    print("\n使用的源文档：")
    for i, doc in enumerate(result["source_documents"], 1):
        print(f"\n相关文档 {i}:")
        print(f"长度：{len(doc.page_content)} 字符")
        print(f"内容片段：{doc.page_content[:150]}...")
        print("---")


/var/folders/bp/x7mjxntx7k75gqf0w3_nsztc0000gn/T/ipykernel_72411/1595382956.py:48: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(



问题：游戏中悟空有哪些形态变化？


/var/folders/bp/x7mjxntx7k75gqf0w3_nsztc0000gn/T/ipykernel_72411/1595382956.py:88: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  result = qa_chain({"query": question})



回答：根据上下文信息，游戏中悟空拥有以下形态变化：
1. **金刚形态**：侧重力量型打击，带来压倒性的破坏力。
2. **魔佛形态**：专注法术攻击，能释放强大的法术伤害。

使用的源文档：

相关文档 1:
长度：444 字符
内容片段：《灭神纪∙猢狲》是一款动作角色扮演游戏。游戏背景设定在架空的神话世界中。玩家将扮演齐天大圣孙悟空，在充满东方神话元素的世界中展开冒险。游戏的战斗系统极具特色，采用了独特的"变身系统"。悟空可以在战斗中变换不同形态。每种形态都有其独特的战斗风格和技能组合。金刚形态侧重力量型打击，带来压倒性的破坏力。魔...
---

问题：游戏的画面风格是怎样的？

回答：游戏的画面表现极具东方美学特色，场景融合了水墨画风格，将山川、建筑等元素完美呈现。战斗特效既有中国传统文化元素，又具备现代游戏的视觉震撼力。

使用的源文档：

相关文档 1:
长度：444 字符
内容片段：《灭神纪∙猢狲》是一款动作角色扮演游戏。游戏背景设定在架空的神话世界中。玩家将扮演齐天大圣孙悟空，在充满东方神话元素的世界中展开冒险。游戏的战斗系统极具特色，采用了独特的"变身系统"。悟空可以在战斗中变换不同形态。每种形态都有其独特的战斗风格和技能组合。金刚形态侧重力量型打击，带来压倒性的破坏力。魔...
---


### 前后向扩展上下文

In [ ]:
# 临时绕过 torch.load 安全检查（仅用于开发环境）
# 注意：这是一个临时解决方案，生产环境应该升级 torch 到 2.6+ 或使用 safetensors 格式
import transformers.utils.import_utils
import transformers.modeling_utils

def dummy_check():
    pass

transformers.utils.import_utils.check_torch_load_is_safe = dummy_check
transformers.modeling_utils.check_torch_load_is_safe = dummy_check


from llama_index.core import VectorStoreIndex, StorageContext, Document, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.postprocessor import PrevNextNodePostprocessor, AutoPrevNextNodePostprocessor
from llama_index.llms.deepseek import DeepSeek
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import os
os.environ["DEEPSEEK_API_KEY"] = "sk-34b4****c05bb4f"


# 配置全局设置
Settings.llm = DeepSeek(model="deepseek-chat", temperature=0.1)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-zh")
Settings.node_parser = SentenceSplitter()
# 准备游戏剧情文本
game_story = """悟空初醒时，发现自己被困在一座古老的山洞中。记忆模糊的他只记得自己是齐天大圣孙悟空，却想不起为何会在此处。洞中有一面破碎的镜子，透过镜子他看到自己伤痕累累，昔日的金箍棒也只剩下一截断柄。离开山洞后，悟空遇到了一位神秘的老僧。老僧告诉他，这里是“幻界”，是介于现实与虚幻之间的特殊空间。500年前，天庭遭遇了前所未有的浩劫，众神陨落，天界崩塌。当时正在大闹天宫的悟空也被卷入其中，失去了大部分法力和记忆，被封印在这个世界。老僧建议悟空去寻找散落在幻界各处的记忆碎片。第一站是位于东方的忘川寺，那里供奉着一面记忆之镜，或许能帮他找回部分记忆。然而，忘川寺已被一群邪魔占领，悟空需要先击败它们。在忘川寺，悟空通过记忆之镜看到了天庭浩劫的部分场景。原来是一个神秘的古老势力在背后操纵，他们利用了“众生之愿”的力量，扭曲了天地规则。当时的悟空虽然强大，却也无法阻止灾难的发生。获得这些记忆后，老僧告诉悟空下一站应该前往西方的业火山。那里有一支蜕变的魔族，他们掌握着更多真相。但业火山常年被熊熊烈火包围，普通生灵难以靠近。悟空需要先找到传说中的三昧火甲，才能安全进入。在寻找三昧火甲的过程中，悟空遇到了昔日的好友妖王。妖王告诉他，天庭崩塌后，六界秩序大乱，各方势力纷纷崛起。有的打着重建天庭的旗号，有的则想建立全新的秩序。一场更大的劫难正在酝酿。获得三昧火甲后，悟空成功潜入业火山。在与魔族首领的对决中，他终于想起了更多真相。原来那个古老势力的目标并非简单的破坏，而是想要重塑整个世界的规则。他们认为现有的秩序存在根本缺陷，导致众生皆苦。回到老僧身边，悟空表示要集结各方力量对抗那个幕后势力。老僧却告诉他，事情可能没有表面看起来那么简单。是否应该重塑世界秩序，这个问题并没有标准答案。建议悟空继续寻找更多真相，再做决定。悟空决定启程前往南方的沉星海。传说那里有一座古老的图书馆，收藏着关于世界起源的众多典籍。然而，在他出发前，幻界突然发生剧烈震动，似乎有什么巨大的变故即将发生……"""
# 创建Document对象
documents = [Document(text=game_story)]
# 构建文档存储和索引，并使用Settings中的node_parser解析文档
nodes = Settings.node_parser.get_nodes_from_documents(documents)
# 创建文档存储并添加节点
docstore = SimpleDocumentStore()
docstore.add_documents(nodes)
# 创建存储上下文
storage_context = StorageContext.from_defaults(docstore=docstore)
# 构建向量索引
index = VectorStoreIndex(nodes, storage_context=storage_context)
# 创建不同的查询引擎
# 基础查询引擎
base_engine = index.as_query_engine(
    similarity_top_k=1,
    response_mode="tree_summarize"
)
# 带固定前后文的查询引擎
prev_next_engine = index.as_query_engine(
    similarity_top_k=1,
    node_postprocessors=[
        PrevNextNodePostprocessor(docstore=docstore, num_nodes=2)
    ],
    response_mode="tree_summarize"
)
# 带自动前后文的查询引擎
auto_engine = index.as_query_engine(
    similarity_top_k=1,
    node_postprocessors=[
        AutoPrevNextNodePostprocessor(
            docstore=docstore,
            num_nodes=3,
            verbose=True
        )
    ],
    response_mode="tree_summarize"
)
# 测试不同类型的问题及不同的查询引擎
test_questions = [
    "悟空从忘川寺获得记忆后发生了什么？",  # 应该找后文
    "悟空是如何到达业火山的？",  # 应该找前文
    "悟空为什么会在山洞中醒来？",  # 应该找前文
]
print("=== 基础查询引擎的结果 ===")
for question in test_questions:
    print(f"\n问题：{question}")
    response = base_engine.query(question)
    print(f"回答：{response}\n")
    print("-" * 50)
print("\n=== 固定前后文查询引擎的结果 ===")
for question in test_questions:
    print(f"\n问题：{question}")
    response = prev_next_engine.query(question)
    print(f"回答：{response}\n")
    print("-" * 50)
print("\n=== 自动前后文查询引擎的结果 ===")
for question in test_questions:
    print(f"\n问题：{question}")
    response = auto_engine.query(question)
    print(f"回答：{response}\n")
print("-" * 50)


=== 基础查询引擎的结果 ===

问题：悟空从忘川寺获得记忆后发生了什么？
回答：悟空从忘川寺获得记忆后，老僧建议他前往西方的业火山，那里有一支蜕变的魔族掌握更多真相。为了进入被烈火包围的业火山，悟空需要先找到传说中的三昧火甲。在寻找过程中，他遇到了昔日好友妖王，得知天庭崩塌后六界秩序大乱，各方势力崛起，一场更大的劫难正在酝酿。获得三昧火甲后，悟空潜入业火山，在与魔族首领的对决中想起了更多真相，了解到古老势力的目标是重塑世界规则。回到老僧身边后，悟空表示要集结力量对抗，但老僧提醒他事情可能更复杂，建议继续寻找真相再做决定。悟空随后决定启程前往南方的沉星海。

--------------------------------------------------

问题：悟空是如何到达业火山的？
回答：悟空在获得三昧火甲后，成功抵达了业火山。

--------------------------------------------------

问题：悟空为什么会在山洞中醒来？
回答：悟空在山洞中醒来是因为他在天庭浩劫中被卷入，失去了大部分法力和记忆，随后被封印在幻界这个特殊空间里。

--------------------------------------------------

=== 固定前后文查询引擎的结果 ===

问题：悟空从忘川寺获得记忆后发生了什么？
回答：悟空从忘川寺获得记忆后，老僧指引他前往西方的业火山，那里有一支魔族掌握更多真相。为了进入被烈火包围的业火山，悟空需要先找到三昧火甲。在寻找过程中，他遇到了妖王，得知天庭崩塌后六界秩序大乱，各方势力崛起，一场更大的劫难正在酝酿。获得三昧火甲后，悟空潜入业火山，在与魔族首领的对决中想起了更多真相，了解到幕后古老势力的目标是重塑世界秩序。回到老僧身边后，悟空打算集结力量对抗，但老僧提醒他事情复杂，重塑秩序并无标准答案，建议继续寻找真相。悟空于是决定前往南方的沉星海，但在出发前，幻界发生了剧烈震动，似乎预示重大变故。

--------------------------------------------------

问题：悟空是如何到达业火山的？
回答：悟空在获得三昧火甲后，成功抵达了业火山。

------------------------------------------------

## 构建有层次的索引

### 双层索引-Milvus

In [23]:
# 双层检索-富豪榜 - 需要pip install openpyxl
import os
from dotenv import load_dotenv
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from pymilvus import MilvusClient, DataType, FieldSchema, CollectionSchema
import logging

# 设置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 加载环境变量
load_dotenv()

# 初始化嵌入模型
embedding_function = SentenceTransformer(
    'BAAI/bge-m3',
    device='cuda:0' if torch.cuda.is_available() else 'cpu',
    trust_remote_code=True
)

# 连接到Milvus
client = MilvusClient("richman_bge_m3_v2.db")

# 1. 创建summary向量数据库
summary_collection_name = "billionaires_summary"
summary_fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=1024),
    FieldSchema(name="table_name", dtype=DataType.VARCHAR, max_length=100)
]

summary_schema = CollectionSchema(summary_fields, "富豪榜年份摘要")
if not client.has_collection(summary_collection_name):
    client.create_collection(
        collection_name=summary_collection_name,
        schema=summary_schema
    )

# 2. 创建details向量数据库
details_collection_name = "billionaires_details"
details_fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=1024),
    FieldSchema(name="table_name", dtype=DataType.VARCHAR, max_length=100),
    FieldSchema(name="content", dtype=DataType.VARCHAR, max_length=10000)  # 存储整个表格内容
]

details_schema = CollectionSchema(details_fields, "富豪榜详细信息")
if not client.has_collection(details_collection_name):
    client.create_collection(
        collection_name=details_collection_name,
        schema=details_schema
    )

# 3. 加载Excel文件并准备数据
excel_file = "90-文档-Data/复杂PDF/十大富豪/世界十大富豪.xlsx"

# 读取Excel文件中的所有sheet并插入数据
with pd.ExcelFile(excel_file) as xls:
    for sheet_name in xls.sheet_names:
        try:
            #df = pd.read_excel(xls, sheet_name=sheet_name)
            df = pd.read_excel(xls, sheet_name=sheet_name, engine='openpyxl')
            logging.info(f"正在处理sheet: {sheet_name}")
            
            # 插入summary数据 - 只存储表名
            summary_embedding = embedding_function.encode([sheet_name])[0]
            
            client.insert(
                collection_name=summary_collection_name,
                data=[{
                    "vector": summary_embedding.tolist(),
                    "table_name": sheet_name
                }]
            )
            
            # 插入details数据 - 存储整个表格内容
            # 将整个DataFrame转换为字符串
            table_content = df.to_string(index=False)
            detail_embedding = embedding_function.encode([table_content])[0]
            
            client.insert(
                collection_name=details_collection_name,
                data=[{
                    "vector": detail_embedding.tolist(),
                    "table_name": sheet_name,
                    "content": table_content
                }]
            )
            
            logging.info(f"成功处理sheet: {sheet_name}")
            
        except Exception as e:
            logging.error(f"处理sheet {sheet_name} 时出错: {str(e)}")
            logging.error(f"错误详情: {e.__class__.__name__}")
            continue

# 4. 创建索引
# 删除已存在的索引（如果有）
try:
    client.drop_index(collection_name=summary_collection_name, index_name="vector")
except Exception as e:
    logging.warning(f"删除summary索引时出错: {str(e)}")

try:
    client.drop_index(collection_name=details_collection_name, index_name="vector")
except Exception as e:
    logging.warning(f"删除details索引时出错: {str(e)}")

# 创建新索引
try:
    # 使用prepare_index_params方法创建索引参数
    summary_index_params = client.prepare_index_params()
    summary_index_params.add_index(
        field_name="vector",  # 指定要为哪个字段创建索引，这里是向量字段
        index_type="IVF_FLAT",  # 索引类型
        metric_type="COSINE",  # 使用余弦相似度作为向量相似度度量方式
        params={"nlist": 1024}  # 索引参数
    )
    
    client.create_index(
        collection_name=summary_collection_name,
        index_params=summary_index_params
    )
    logging.info("成功创建summary索引")
except Exception as e:
    logging.error(f"创建summary索引时出错: {str(e)}")

try:
    # 使用prepare_index_params方法创建索引参数
    details_index_params = client.prepare_index_params()
    details_index_params.add_index(
        field_name="vector",  # 指定要为哪个字段创建索引，这里是向量字段
        index_type="IVF_FLAT",  # 索引类型
        metric_type="COSINE",  # 使用余弦相似度作为向量相似度度量方式
        params={"nlist": 1024}  # 索引参数
    )
    
    client.create_index(
        collection_name=details_collection_name,
        index_params=details_index_params
    )
    logging.info("成功创建details索引")
except Exception as e:
    logging.error(f"创建details索引时出错: {str(e)}")

# 加载集合以使索引生效
try:
    client.load_collection(summary_collection_name)
    client.load_collection(details_collection_name)
    logging.info("成功加载集合")
except Exception as e:
    logging.error(f"加载集合时出错: {str(e)}")

def search_relevant_table(question):
    # 第一层检索：在summary集合中搜索最相关的sheet
    query_embedding = embedding_function.encode([question])[0]
    
    summary_results = client.search(
        collection_name=summary_collection_name,
        data=[query_embedding.tolist()],
        limit=1,
        output_fields=["table_name"],
        search_params={
            "metric_type": "COSINE",
            "params": {"nprobe": 10}
        }
    )
    
    if not summary_results or not summary_results[0]:
        return None, None
    
    matched_table = summary_results[0][0]['entity']['table_name']
    
    # 第二层检索：在details集合中搜索具体信息
    details_results = client.search(
        collection_name=details_collection_name,
        data=[query_embedding.tolist()],
        filter=f"table_name == '{matched_table}'",
        limit=1,
        output_fields=["content"],
        search_params={
            "metric_type": "COSINE",
            "params": {"nprobe": 10}
        }
    )
    
    if not details_results or not details_results[0]:
        return None, None
    
    return matched_table, details_results[0][0]['entity']['content']

def generate_answer(question):
    # 检索相关信息
    table_name, content = search_relevant_table(question)
    
    if not table_name or not content:
        return "抱歉，没有找到相关信息。"
    
    # 构建提示词
    prompt = f"""根据以下表格信息回答问题：

表格名称：{table_name}

表格内容：
{content}

问题：{question}

请基于以上信息给出详细回答："""

    # 使用DeepSeek生成答案
    from openai import OpenAI
    client = OpenAI(
        api_key="sk-34b4f11441fc4182a2509ed6dc05bb4f",
        base_url="https://api.deepseek.com/v1"
    )

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[{
            "role": "user",
            "content": prompt
        }],
        max_tokens=1024
    )
    
    return response.choices[0].message.content

# 测试示例
if __name__ == "__main__":
    test_question = "2023年世界首富是谁？他的财富是多少？"
    answer = generate_answer(test_question)
    print(f"问题：{test_question}")
    print(f"答案：{answer}") 

问题：2023年世界首富是谁？他的财富是多少？
答案：根据表格信息，**2023年世界首富是伯纳德·阿尔诺及其家族（Bernard Arnault & family）**，其财富净值为 **2110亿美元**。  

**详细说明**：  
1. 表格中排名第一（No. 1）的即为伯纳德·阿尔诺，他是法国奢侈品巨头LVMH集团的掌门人。  
2. 其财富净值明确列为“$211 billion”，即2110亿美元。  
3. 表格标题为“2023年10大首富”，因此该数据反映的是2023年的排名情况。  

**答案总结**：  
- **姓名**：伯纳德·阿尔诺及其家族  
- **财富**：2110亿美元  
- **国籍**：法国  
- **财富来源**：LVMH（酩悦·轩尼诗－路易·威登集团）


### 双层索引-PandasNode

In [ ]:
# 双层检索-富豪榜 - 需要pip install openpyxl

# 临时绕过 torch.load 安全检查（仅用于开发环境）
import transformers.utils.import_utils
import transformers.modeling_utils

def dummy_check():
    pass

transformers.utils.import_utils.check_torch_load_is_safe = dummy_check
transformers.modeling_utils.check_torch_load_is_safe = dummy_check

import os
from dotenv import load_dotenv
import pandas as pd
import logging
from llama_index.core import Document, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

from llama_index.llms.deepseek import DeepSeek
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# 设置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 加载环境变量
load_dotenv()

import os
os.environ["DEEPSEEK_API_KEY"] = "sk-34b***c05bb4f"

# 设置全局设置 - 使用 DeepSeek 和 HuggingFace
Settings.llm = DeepSeek(model="deepseek-chat", temperature=0.1)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-zh")

# 3. 加载Excel文件并准备数据
excel_file = "90-文档-Data/复杂PDF/十大富豪/世界十大富豪.xlsx"

# 初始化Node Parser
node_parser = SentenceSplitter(
    chunk_size=1024,  # 每个chunk的大小
    chunk_overlap=20,  # chunk之间的重叠大小
    include_metadata=True  # 包含元数据
)

# 存储所有表格的DataFrame和查询引擎
table_dfs = []
df_query_engines = []
documents = []

# 读取Excel文件中的所有sheet并插入数据
with pd.ExcelFile(excel_file) as xls:
    for sheet_name in xls.sheet_names:
        try:
            df = pd.read_excel(xls, sheet_name=sheet_name)
            logging.info(f"正在处理sheet: {sheet_name}")
            
            # 将DataFrame转换为字符串
            table_content = df.to_string(index=False)
            
            # 创建Document对象
            doc = Document(
                text=table_content,
                metadata={"table_name": sheet_name}
            )
            documents.append(doc)
            
            # 存储DataFrame和创建查询引擎
            table_dfs.append(df)
            df_query_engine = PandasQueryEngine(df, llm=Settings.llm)
            df_query_engines.append(df_query_engine)
            
            logging.info(f"成功处理sheet: {sheet_name}")
            
        except Exception as e:
            logging.error(f"处理sheet {sheet_name} 时出错: {str(e)}")
            logging.error(f"错误详情: {e.__class__.__name__}")
            continue

# 创建IndexNode对象
summaries = [
    f"This node provides information about the world's richest billionaires in {sheet_name}"
    for sheet_name in xls.sheet_names
]

df_nodes = [
    IndexNode(text=summary, index_id=f"pandas{idx}") # 每个表的细节
    for idx, summary in enumerate(summaries)
]

# 创建查询引擎映射
df_id_query_engine_mapping = {
    f"pandas{idx}": df_query_engine
    for idx, df_query_engine in enumerate(df_query_engines)
}

# 创建向量索引 - 只使用 df_nodes，不包含原始文档
vector_index = VectorStoreIndex(df_nodes)
vector_retriever = vector_index.as_retriever(similarity_top_k=1)

# 创建递归检索器
recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    query_engine_dict=df_id_query_engine_mapping,
    verbose=True,
)

# 创建响应合成器
response_synthesizer = get_response_synthesizer(response_mode="compact")

# 创建查询引擎
query_engine = RetrieverQueryEngine.from_args(
    recursive_retriever, response_synthesizer=response_synthesizer
)

def generate_answer(question):
    # 使用查询引擎生成答案
    response = query_engine.query(question)
    return str(response)

# 测试示例
if __name__ == "__main__":
    test_question = "2020年世界首富是谁？他的财富是多少？"
    answer = generate_answer(test_question)
    print(f"问题：{test_question}")
    print(f"答案：{answer}") 

Retrieving with query id None: 2020年世界首富是谁？他的财富是多少？
Retrieved node with id, entering: pandas2
Retrieving with query id pandas2: 2020年世界首富是谁？他的财富是多少？
Got response: Name           Jeff Bezos
Net_Worth    $177 billion
Name: 0, dtype: object
问题：2020年世界首富是谁？他的财富是多少？
答案：2020年世界首富是杰夫·贝索斯，他的财富为1770亿美元。


### 双层索引 FAISS

In [25]:
# 临时绕过 torch.load 安全检查（仅用于开发环境）
import transformers.utils.import_utils
import transformers.modeling_utils

def dummy_check():
    pass

transformers.utils.import_utils.check_torch_load_is_safe = dummy_check
transformers.modeling_utils.check_torch_load_is_safe = dummy_check

import os
from dotenv import load_dotenv
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from openai import OpenAI

# 加载环境变量
load_dotenv()

# 1. 准备表格说明数据
table_descriptions = [
    "2023年世界十大富豪榜单，展示了当年全球最富有的十位富豪及其财富情况。",
    "2022年世界十大富豪榜单，记录了当年全球最富有的十位富豪及其财富情况。",
    "2021年世界十大富豪榜单，展示了当年全球最富有的十位富豪及其财富情况。",
    "2020年世界十大富豪榜单，记录了当年全球最富有的十位富豪及其财富情况。",
    "2019年世界十大富豪榜单，展示了当年全球最富有的十位富豪及其财富情况。"
]

# 2. 设置第一层嵌入模型（用于匹配年份）
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
desc_embeddings = model.encode(table_descriptions)

# 3. 创建第一层向量存储
dimension = desc_embeddings.shape[1]
desc_index = faiss.IndexFlatL2(dimension)
desc_index.add(desc_embeddings.astype('float32'))

# 4. 加载Excel文件并准备第二层数据
excel_file = "90-文档-Data/复杂PDF/十大富豪/世界十大富豪.xlsx"
all_tables_data = {}
sheet_names_list = []

# 读取Excel文件中的所有sheet
with pd.ExcelFile(excel_file) as xls:
    sheet_names_list = xls.sheet_names
    for sheet_name in sheet_names_list:
        df = pd.read_excel(xls, sheet_name=sheet_name)
        # 将DataFrame转换为文本格式
        table_text = df.to_string(index=False)
        all_tables_data[sheet_name] = table_text

# 5. 创建第二层向量存储
table_embeddings = model.encode(list(all_tables_data.values()))
table_index = faiss.IndexFlatL2(dimension)
table_index.add(table_embeddings.astype('float32'))

def search_relevant_table(question):
    # 第一层检索：匹配年份描述
    query_embedding = model.encode([question])[0]
    distances, indices = desc_index.search(
        np.array([query_embedding]).astype('float32'), 
        k=1
    )
    matched_index = indices[0][0]
    
    # 获取对应的 sheet 名称
    matched_sheet_name = sheet_names_list[matched_index]
    
    # 返回匹配的描述和表格数据
    return table_descriptions[matched_index], all_tables_data[matched_sheet_name]

def generate_answer(question):
    # 检索相关信息
    year_context, table_context = search_relevant_table(question)
    
    # 构建提示词
    prompt = f"""根据以下参考信息回答问题：
    
年份信息：{year_context}

相关数据：
{table_context}

问题：{question}

请基于以上信息给出详细回答："""

    # 使用DeepSeek生成答案
    client = OpenAI(
        api_key="sk-34b4f11441fc4182a2509ed6dc05bb4f",
        base_url="https://api.deepseek.com/v1"
    )

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[{
            "role": "user",
            "content": prompt
        }],
        max_tokens=1024
    )
    
    return response.choices[0].message.content

# 测试示例
if __name__ == "__main__":
    test_question = "2023年世界首富是谁？他的财富是多少？"
    answer = generate_answer(test_question)
    print(f"问题：{test_question}")
    print(f"答案：{answer}") 

问题：2023年世界首富是谁？他的财富是多少？
答案：根据提供的2023年世界十大富豪榜单信息，**2023年世界首富是伯纳德·阿尔诺及其家族（Bernard Arnault & family）**。  

他的财富净值为 **2110亿美元**（$211 billion），年龄74岁，国籍为法国，财富主要来源于奢侈品集团LVMH。  

因此，答案是：  
- **首富**：伯纳德·阿尔诺及其家族  
- **财富**：2110亿美元
